In [32]:
import sqlite3
listaInfoLibros = [(1,u'El Quijote', u'Miguel de Cervantes',u'Alianza',u'LIBRO',u'1988-06-11',u'España',12),
                    (2,u'Marina', u'Carlos Ruiz Zafón',u'Edebé',u'CD',u'2003-05-10',u'España',18.95),
                    (3,u'La hoguera de las vanidades',u'Tom Wolfe',u'RBA editores',u'DVD',u'2005-11-09',u'USA',22.25),
                    (4,u'Los pilares de la Tierra',u'Ken Follet',u'Faber',u'LIBRO',u'2014-12-01',u'USA',12.95),
                    (5,u'Otelo',u'William Shakespeare',u'Anaya',u'LIBRO',u'2013-04-11',u'Inglaterra',14.95),
                    (6,u'Rimas y Leyendas',u'Gustavo Adolfo Becquer',u'Roca',u'LIBRO',u'2008-01-08',u'España',25.95),
                    (7,u'Poesía',u'Juan Ramón Jimenez',u'P&J',u'LIBRO',u'2002-04-07',u'España',10.95)]

listaInfoCompradores = [(1,u'Juan Miedo',u'1955-10-23',u'608900890',u'La isla del tesoro,33',u'Getafe',u'Buen comprador'),
                        (2,u'Pepe Pepino',u'1961-12-13',u'607899005',u'Plaza Mayor,56',u'Pozuelo',u''),
                        (3,u'Pepe Mur',u'1976-04-02',u'917895679',u'Esparteros,5',u'Getafe',u''),
                        (4,u'Mohamed Alí',u'1968-11-12',u'609440567',u'Juan sin miedo,4',u'Pozuelo',u'Le gusta la ciencia ficción'),
                        (5,u'Alfredo Mesa',u'1986-08-17',u'690890456',u'Gran vía,56',u'Getafe',u'Le gustan los ensayos'),
                        (6,u'Pedro Reyes',u'1957-08-25',u'917890056',u'Plaza de España,34',u'Pozuelo',u'Le gusta la historia'),
                        (7,u'Isabel Olvido',u'1977-07-20',u'915678900',u'Principal,3',u'Getafe',u'Le gusta la novela de terror'),
                        (8,u'Mariano Calcetines',u'1996-11-09',u'634567876',u'Aviación,34',u'Getafe',u''),
                        (9,u'María Calero',u'1984-11-08',u'645666900',u'Río Ebro,4',u'Las Rozas',u'') ]

listaInfoCompras = [(1,9,7), (2,9,3), (3,8,2), (4,7,1), (5,8,1), (6,1,1), (7,7,1), (8,6,2), (9,3,5), (10,3,1), (11,3,2)]

conn=sqlite3.connect("Libreria.sqlite3")

cur=conn.cursor()
cur.execute("DROP TABLE IF EXISTS Libros")
cur.execute("DROP TABLE IF EXISTS Compradores")
cur.execute("DROP TABLE IF EXISTS Compras")
cur.execute(u"""
        CREATE TABLE Libros(
            registro INTEGER(4) PRIMARY KEY NOT NULL UNIQUE,
            titulo VARCHAR(35) NOT NULL DEFAULT '' UNIQUE,
            escritor VARCHAR(35) NOT NULL DEFAULT '',
            editorial VARCHAR(20) NOT NULL DEFAULT '',
            soporte VARCHAR(35) NOT NULL DEFAULT 'LIBRO',
            fecha_entrada DATE NOT NULL DEFAULT '0000-00-00' UNIQUE,
            pais VARCHAR(20) NOT NULL DEFAULT '',
            importe DECIMAL(8,2) NOT NULL DEFAULT 0.0,
            anotaciones BLOB
            )""")
cur.execute(u"""
        CREATE TABLE Compradores(
            registro INTEGER(4) PRIMARY KEY NOT NULL UNIQUE,
            nombre VARCHAR(35) NOT NULL DEFAULT '',
            fecha_nacimiento DATE NOT NULL DEFAULT '0000-00-00',
            telefono VARCHAR(10) DEFAULT NULL,
            domicilio  VARCHAR(35) DEFAULT NULL,
            poblacion  VARCHAR(25) DEFAULT NULL,
            anotaciones  TEXT
            )""")

cur.execute(u"""
        CREATE TABLE Compras(
            registro INTEGER(4) PRIMARY KEY NOT NULL UNIQUE,
            id_comprador INTEGER(4) NOT NULL DEFAULT '',
            id_libro INTEGER(4) NOT NULL DEFAULT ''
            )""")

cur.executemany("""
        INSERT INTO Libros (registro, titulo, escritor, editorial, soporte, fecha_entrada,pais,importe)
        VALUES ( ?, ?, ?, ? ,?, ? ,?, ?) """, listaInfoLibros)
cur.executemany("""
        INSERT INTO Compradores (registro, nombre, fecha_nacimiento,telefono, domicilio,poblacion,anotaciones)
        VALUES ( ?, ?, ?, ?, ?, ? ,?) """,listaInfoCompradores)
cur.executemany('INSERT INTO Compras (registro, id_comprador, id_libro) VALUES ( ?, ?, ?) ',listaInfoCompras)
#consultas
"""
cur.execute('SELECT pais,count(pais) as libros_vendidos,sum(importe) as total_ventas from Libros l, Compras c where c.id_libro=l.registro group by pais order by total_ventas DESC')
for (pais, libros,importe) in cur.fetchall():
    print ("Pais:", pais)
    print ("Num libros:", libros)
    print ("Total ganado:", importe)
    print ("############################")

cur.execute('SELECT poblacion,avg(importe) as media from Compradores c ,Compras cl ,Libros l where l.registro=cl.id_libro AND c.registro=cl.id_comprador group by poblacion order by media DESC')
for (poblacion,importe) in cur.fetchall():
    print ("Poblacion:", poblacion)
    print ("Importe medio:", importe)
    print ("############################")

cur.execute('SELECT registro,id_comprador,id_libro from Compras where registro=11 OR registro=10')
print(cur.fetchall())
cur.execute('UPDATE Compras set id_comprador=?, id_libro=? WHERE registro=?', [3,3,10])
cur.execute('UPDATE Compras set id_comprador=?, id_libro=? WHERE registro=?', [3,7,11])
conn.commit()
cur.execute('SELECT registro,id_comprador,id_libro from Compras where registro=11 OR registro=10')
print(cur.fetchall())

cur.execute('SELECT soporte,avg(importe) as media from Libros group by soporte')
for (soporte,media) in cur.fetchall():
    print ("Soporte:", soporte)
    print ("Media:", media)
    print ("############################")

cur.execute('select * from compradores')
print(cur.fetchall())
cur.execute('DELETE FROM Compradores WHERE registro IN (SELECT registro FROM Compradores WHERE registro NOT IN ( SELECT c.registro FROM Compradores c, Compras cl WHERE c.registro = cl.id_comprador))')
cur.execute('select * from compradores')
print(cur.fetchall())
"""
cur.close()

[(1, 'Juan Miedo', '1955-10-23', '608900890', 'La isla del tesoro,33', 'Getafe', 'Buen comprador'), (2, 'Pepe Pepino', '1961-12-13', '607899005', 'Plaza Mayor,56', 'Pozuelo', ''), (3, 'Pepe Mur', '1976-04-02', '917895679', 'Esparteros,5', 'Getafe', ''), (4, 'Mohamed Alí', '1968-11-12', '609440567', 'Juan sin miedo,4', 'Pozuelo', 'Le gusta la ciencia ficción'), (5, 'Alfredo Mesa', '1986-08-17', '690890456', 'Gran vía,56', 'Getafe', 'Le gustan los ensayos'), (6, 'Pedro Reyes', '1957-08-25', '917890056', 'Plaza de España,34', 'Pozuelo', 'Le gusta la historia'), (7, 'Isabel Olvido', '1977-07-20', '915678900', 'Principal,3', 'Getafe', 'Le gusta la novela de terror'), (8, 'Mariano Calcetines', '1996-11-09', '634567876', 'Aviación,34', 'Getafe', ''), (9, 'María Calero', '1984-11-08', '645666900', 'Río Ebro,4', 'Las Rozas', '')]
[(1, 'Juan Miedo', '1955-10-23', '608900890', 'La isla del tesoro,33', 'Getafe', 'Buen comprador'), (3, 'Pepe Mur', '1976-04-02', '917895679', 'Esparteros,5', 'Getafe'